In [15]:
#Q1
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,KFold

In [5]:
df=pd.read_csv("USA_Housing.csv")
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [6]:
X=df.drop(columns=['Price'])
y=df['Price']

In [8]:
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [9]:
def add_bias(X):
    return np.hstack((np.ones((X.shape[0], 1)), X))

In [22]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold=0
betas=[]
r2_scores=[]

In [23]:
for train_idx,test_idx in kf.split(X):
    fold+=1
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train_bias = add_bias(X_train)
    X_test_bias = add_bias(X_test)
    beta = np.linalg.pinv(X_train_bias) @ y_train
    betas.append(beta)
    y_pred = X_test_bias @ beta
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    print(f"Fold {fold} --> R2 = {r2:.4f}")
best_fold_index = np.argmax(r2_scores)
best_beta = betas[best_fold_index]
print("\nBest Fold:", best_fold_index + 1)
print("Best R2:", r2_scores[best_fold_index])

Fold 1 --> R2 = 0.9180
Fold 2 --> R2 = 0.9146
Fold 3 --> R2 = 0.9116
Fold 4 --> R2 = 0.9193
Fold 5 --> R2 = 0.9244

Best Fold: 5
Best R2: 0.9243869413350316


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train_bias = add_bias(X_train)
X_test_bias = add_bias(X_test)

In [13]:
y_pred_best_beta = X_test_bias @ best_beta
final_r2 = r2_score(y_test, y_pred_best_beta)

In [26]:
print("\nR2 using best fold beta :", final_r2)


R2 using best fold beta : 0.9147458156636434


In [27]:
#Q2
LEARNING_RATES = [0.001, 0.01, 0.1, 1]

In [32]:
def gradient_descent(X_bias, y, lr, n_iter):
    y = y.values if isinstance(y, pd.Series) else y  
    m, n = X_bias.shape
    beta = np.zeros(n)
    
    for i in range(n_iter):
        preds = X_bias @ beta
        error = preds - y
        grad = (1/m) * (X_bias.T @ error)
        beta = beta - lr * grad
        
    return beta

In [33]:
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, test_size=0.20, random_state=42)

In [34]:
X_train_bias = add_bias(X_train)
X_val_bias   = add_bias(X_val)
X_test_bias  = add_bias(X_test)

In [35]:
results=[]
for lr in LEARNING_RATES:
    beta = gradient_descent(X_train_bias, y_train, lr, n_iter=1000)
    y_val_pred = X_val_bias @ beta
    y_test_pred = X_test_bias @ beta
    
    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    results.append({
        "lr": lr,
        "beta": beta,
        "r2_val": r2_val,
        "r2_test": r2_test
    })
    print(f"lr={lr:>5}  ->  R2_val = {r2_val:.6f} , R2_test = {r2_test:.6f}")

lr=0.001  ->  R2_val = -0.812520 , R2_test = -0.991365
lr= 0.01  ->  R2_val = 0.909818 , R2_test = 0.914743
lr=  0.1  ->  R2_val = 0.909800 , R2_test = 0.914757
lr=    1  ->  R2_val = 0.909800 , R2_test = 0.914757


In [37]:
best = max(results, key=lambda x: x["r2_val"])
print("\nBest learning rate (by validation R²):", best["lr"])
print("Validation R²:", best["r2_val"])
print("Test R² for that beta:", best["r2_test"])
print("Beta (first 10 elements shown):")
print(np.round(best["beta"].flatten()[:10], 6))


Best learning rate (by validation R²): 0.01
Validation R²: 0.9098183094422969
Test R² for that beta: 0.9147434800538763
Beta (first 10 elements shown):
[1232562.512549  230048.766647  163686.935036  121406.941079
    3117.473639  150655.974597]
